In [3]:
# Imports modules
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
batch_size = 5
num_features = ()

In [2]:
def create_model():
    model = Sequential()
    model.add(LSTM(128, activation = 'relu', dropout = 0.2,))
    model.add(LSTM(64, activation = 'relu', dropout = 0.2,))
    model.add(LSTM(32, activation = 'relu', dropout = 0.2,))
    model.add(Dense(1, activation = "relu"))
    return model